# Setup

In [ ]:
from hello_world.domain      import Timetable, ShiftAssignment, Shift, TA, ConstraintParameters
from hello_world.utils       import print_ta_availability, initialize_logger, id_generator
from hello_world.solver      import TimetableSolverWithSolverManager

from typing import List, Tuple, Dict

import datetime as  dt

In [3]:
# Constants
AFTERNOON_START_TIME = dt.time(14, 30)
AFTERNOON_END_TIME   = dt.time(17, 30)

EVENING_START_TIME =  dt.time(18, 30)
EVENING_END_TIME   =  dt.time(21, 30)

NUM_OF_WEEKS = 1

DEFAULT_MIN_SHIFTS_PER_WEEK = 0
DEFAULT_MAX_SHIFTS_PER_WEEK = 2

DEFAULT_REQUIRED_SHIFTS_PER_SEMESTER = DEFAULT_MAX_SHIFTS_PER_WEEK * NUM_OF_WEEKS

# Logger Initialization
logger = initialize_logger()
logger.info("Starting test for 2DX3 timetable in 2026")

...
Logger initialized
	@ 2026-01-11 17:25:22
	Constraint version: default
	Data:               default
...
Starting test for 2DX3 timetable in 2026


# Shifts

## Enter Data

In [4]:
# Shift Data (<Series>, <Start Time>, <End Time>, <Required TAs>)
# ----------------------------------------------------------------------
# Monday Shifts
SHIFT_L01 = ("L01", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 3)
SHIFT_L07 = ("L07", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 2)

# Tuesday Shifts
SHIFT_L02 = ("L02", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 3)
SHIFT_L08 = ("L08", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 3)


# Wednesday Shifts
SHIFT_L03 = ("L03", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 3)
SHIFT_L09 = ("L09", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 2)

SHIFT_L06 = ("L06", EVENING_START_TIME, EVENING_END_TIME, 3)

# Thursday Shifts
SHIFT_L04 = ("L04", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 3)
SHIFT_L10 = ("L10", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 2)

# Friday Shifts
SHIFT_L05 = ("L05", AFTERNOON_START_TIME, AFTERNOON_END_TIME, 3)

# Create Shifts
shift_details_list = [ SHIFT_L01, SHIFT_L02, SHIFT_L03, SHIFT_L04, SHIFT_L05,
           SHIFT_L06, SHIFT_L07, SHIFT_L08, SHIFT_L09, SHIFT_L10 ]

## Generate Objects

In [5]:
def generate_shift(shifts: List[Tuple[str, dt.time, dt.time, int]], num_weeks: int) -> Dict[str, List[Shift]]:
    shift_list = {}
    ids = id_generator()
    for item in shifts:
        series, start_time, end_time, required_tas = item
        temp = []
        for i in range(num_weeks):
            shift = Shift(
                id=next(ids),
                series=series,
                day_of_week="NA",
                week_id=i,
                start_time=start_time,
                end_time=end_time,
                required_tas=required_tas
            )
            temp.append(shift)
        shift_list[series] = temp
    return shift_list

def convert_dict_to_list(shift_dict: Dict[str, List[Shift]]) -> List[Shift]:
    shift_list = []
    for key in shift_dict:
        shift_list.extend(shift_dict[key])
    return shift_list

In [6]:
logger.info("Generating shifts for 2DX3 course; details:")
logger.info(f"Shift Details List: {[name for name, _, _, _ in shift_details_list]}")

shifts_dict = generate_shift(shifts=shift_details_list, num_weeks=NUM_OF_WEEKS)
shifts = convert_dict_to_list(shifts_dict)

logger.info(f"Generated {len(shifts)} shifts")

Generating shifts for 2DX3 course; details:
Shift Details List: ['L01', 'L02', 'L03', 'L04', 'L05', 'L06', 'L07', 'L08', 'L09', 'L10']
Generated 10 shifts


# TAs

## Enter Data

In [7]:
ta_data = {
    "TA-1" : {
        "unavailable_shifts": shifts_dict["L01"] + shifts_dict["L02"],
        "max_shifts_per_week": 3
    },
    "TA-2" : {
        "unavailable_shifts": shifts_dict["L03"] + shifts_dict["L04"],
        "max_shifts_per_week": 3
    },
    "TA-3" : {
        "unavailable_shifts": shifts_dict["L05"] + shifts_dict["L06"],
        "max_shifts_per_week": 2
    },
    "TA-3" : {
        "unavailable_shifts": shifts_dict["L05"] + shifts_dict["L06"],
        "max_shifts_per_week": 2
    },
    "TA-3" : {
        "unavailable_shifts": shifts_dict["L05"] + shifts_dict["L06"],
        "max_shifts_per_week": 2
    },
    "TA-4" : {
        "unavailable_shifts": shifts_dict["L05"] + shifts_dict["L06"],
        "max_shifts_per_week": 2
    },
}


## Generate Objects

In [8]:
def create_ta_objects(ta_data: Dict[str, Dict]) -> List[TA]:
    ta_objects = []
    id_gen = id_generator()
    for ta_name, details in ta_data.items():
        ta = TA(
            id=next(id_gen),
            name=ta_name,
            desired=details.get("desired_shifts", []),
            undesired=details.get("undesired_shifts", []),
            unavailable=details.get("unavailable_shifts", []),
            skill_level=details.get("skill_level", 1),
            required_shifts_per_semester=details.get("required_shifts_per_semester", DEFAULT_REQUIRED_SHIFTS_PER_SEMESTER),
            min_shifts_per_week=details.get("min_shifts_per_week", DEFAULT_MIN_SHIFTS_PER_WEEK),
            max_shifts_per_week=details.get("max_shifts_per_week", DEFAULT_MAX_SHIFTS_PER_WEEK),
        )
        ta_objects.append(ta)
    return ta_objects

ta_list = create_ta_objects(ta_data=ta_data)
logger.info("Created TA objects with the following details:")
logger.info(f"\t{len(ta_list)} TA objects")
logger.info(f"\tTA Names: {[ta.name for ta in ta_list]}")

Created TA objects with the following details:
	4 TA objects
	TA Names: ['TA-1', 'TA-2', 'TA-3', 'TA-4']


# Create the problem

In [9]:
def generate_shift_assignments(shifts: List[Shift]) -> List[ShiftAssignment]:
    shift_assignments = []
    id_gen = id_generator()
    for shift in shifts:
        for i in range(shift.required_tas):
            shift_assignment = ShiftAssignment(
                id=next(id_gen),
                shift=shift,
                assigned_ta=None
            )
            shift_assignments.append(shift_assignment)
    return shift_assignments


shift_assignments_unsolved = generate_shift_assignments(shifts=shifts)
len(shift_assignments_unsolved), len(shifts)

(27, 10)

In [10]:
id_gen = id_generator()
problem = Timetable(
    id=next(id_gen),
    shifts=shifts,
    tas=ta_list,
    constraint_parameters=ConstraintParameters(),
    shift_assignments=shift_assignments_unsolved
)
problem

Timetable(id='0', shifts=[Shift(id='0', series='L01', day_of_week='NA', week_id=0, start_time=datetime.time(14, 30), end_time=datetime.time(17, 30), required_tas=3, alias='DEFAULT', shift_date=datetime.date(1900, 1, 1)), Shift(id='1', series='L02', day_of_week='NA', week_id=0, start_time=datetime.time(14, 30), end_time=datetime.time(17, 30), required_tas=3, alias='DEFAULT', shift_date=datetime.date(1900, 1, 1)), Shift(id='2', series='L03', day_of_week='NA', week_id=0, start_time=datetime.time(14, 30), end_time=datetime.time(17, 30), required_tas=3, alias='DEFAULT', shift_date=datetime.date(1900, 1, 1)), Shift(id='3', series='L04', day_of_week='NA', week_id=0, start_time=datetime.time(14, 30), end_time=datetime.time(17, 30), required_tas=3, alias='DEFAULT', shift_date=datetime.date(1900, 1, 1)), Shift(id='4', series='L05', day_of_week='NA', week_id=0, start_time=datetime.time(14, 30), end_time=datetime.time(17, 30), required_tas=3, alias='DEFAULT', shift_date=datetime.date(1900, 1, 1)),

# Solve the problem

In [11]:
solver = TimetableSolverWithSolverManager(constraint_version="tabriz", logger=logger)
# solver = TimetableSolverBlocking(constraint_version="tabriz", logger=logger)

solution = solver.solve_problem(problem=problem)


🚀 === Starting to Solve the Problem ===
	Running a sanity check on the problem...
Performing sanity check on the problem instance...
	(1) Domain-specific sanity checks...
	Problem has 4 TAs and 10 shifts.
	⚠️  SANITY CHECK WARNING [TMTBL.workforce.count]: Total required TAs (27) over the planning problem exceeds anticipated TA duties (10).
		Count_TAs: 4, Num_of_weeks: 1, Total_available_TAs_per_week: 10
		Consider increasing the number of TAs or their maximum shifts per week, or reducing the required TAs per shift.
	⚠️  SANITY CHECK WARNING [TMTBL.shifts.desired]: No TAs have shift series 'L01' in week '0' as desired.
	⚠️  SANITY CHECK WARNING [TMTBL.shifts.desired]: No TAs have shift series 'L02' in week '0' as desired.
	⚠️  SANITY CHECK WARNING [TMTBL.shifts.desired]: No TAs have shift series 'L03' in week '0' as desired.
	⚠️  SANITY CHECK WARNING [TMTBL.shifts.desired]: No TAs have shift series 'L04' in week '0' as desired.
	⚠️  SANITY CHECK WARNING [TMTBL.shifts.unavailable]: Onl